In [ ]:
import tensorflow as tf
print(tf.test.gpu_device_name())
print("TensorFlow version:", tf.__version__)
!nvidia-smi

Daten Importieren

In [ ]:
import gzip
import os
import numpy as np

# Trainingsdaten einlesen
X_train = mnist_images(train_data)
y_train = mnist_labels(train_labels)

# Testdaten einlesen
X_test = mnist_images(test_data)
y_test = mnist_labels(test_labels)

In [7]:
import os
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split



xlsx_path = 'Bildervorbearbeitung/Bilder/xlsx'

# Erstellen Sie einen leeren DataFrame, um die kombinierten Daten aufzunehmen
combined_data = pd.DataFrame(columns=['Bildname', 'Porositaet'])

# Verzeichnispfad, in dem Ihre kleinen Excel-Dateien gespeichert sind
xlsx_path = 'Bildervorbearbeitung/Bilder/xlsx'  

# Durchlaufen Sie die Dateien im Verzeichnis
for filename in os.listdir(xlsx_path):
    if filename.endswith('.xlsx'):  # Stellen Sie sicher, dass es sich um Excel-Dateien handelt
        # Vollständigen Pfad zur aktuellen Datei erstellen
        file_path = os.path.join(xlsx_path, filename)
        
        # Laden Sie die Daten aus der aktuellen Datei in einen DataFrame
        data = pd.read_excel(file_path)
        
        # Fügen Sie die Daten zum kombinierten DataFrame hinzu
        combined_data = combined_data.append(data, ignore_index=True)

# Speichern Sie die kombinierten Daten in einer neuen Excel-Datei oder verwenden Sie sie wie gewünscht
combined_data.to_excel('import_data.xlsx', index=False)

img_folder_path = "Bildervorbearbeitung/Bilder/Zugeschnittene_Bilder"

X = []
y = []

import_data = pd.read_excel('import_data.xlsx')
for index, row in import_data.iterrows():
    img_name = row['Bildname']
    img_path = os.path.join(img_folder_path, img_name)
    img = cv2.imread(img_path)
    X.append(img)
    y.append(row['Porositaet'])

# Aufteilung in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# test_size gibt den Anteil der Testdaten an (hier 20%)
# random_state ist eine Zufallszahl für die Reproduzierbarkeit der Aufteilung



/var/folders/ny/0_rl0ybx6dn5qlc0nwtzw8s00000gn/T/ipykernel_61192/2113659570.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(data, ignore_index=True)
/var/folders/ny/0_rl0ybx6dn5qlc0nwtzw8s00000gn/T/ipykernel_61192/2113659570.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(data, ignore_index=True)
/var/folders/ny/0_rl0ybx6dn5qlc0nwtzw8s00000gn/T/ipykernel_61192/2113659570.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_data = combined_data.append(data, ignore_index=True)
/var/folders/ny/0_rl0ybx6dn5qlc0nwtzw8s00000gn/T/ipykernel_61192/2113659570.py:26: FutureWarning: The frame.append method is deprecated and will be removed from

[array([[[ 36,  36,  36],
        [ 21,  21,  21],
        [ 14,  14,  14],
        ...,
        [228, 228, 228],
        [228, 228, 228],
        [228, 228, 228]],

       [[ 37,  37,  37],
        [ 21,  21,  21],
        [ 12,  12,  12],
        ...,
        [229, 229, 229],
        [231, 231, 231],
        [231, 231, 231]],

       [[ 28,  28,  28],
        [ 19,  19,  19],
        [ 14,  14,  14],
        ...,
        [228, 228, 228],
        [229, 229, 229],
        [230, 230, 230]],

       ...,

       [[230, 230, 230],
        [230, 230, 230],
        [230, 230, 230],
        ...,
        [226, 226, 226],
        [227, 227, 227],
        [229, 229, 229]],

       [[229, 229, 229],
        [228, 228, 228],
        [231, 231, 231],
        ...,
        [227, 227, 227],
        [227, 227, 227],
        [227, 227, 227]],

       [[231, 231, 231],
        [228, 228, 228],
        [231, 231, 231],
        ...,
        [227, 227, 227],
        [226, 226, 226],
        [226, 226, 226]

In [8]:
print(X)
print(y)

[array([[[ 36,  36,  36],
        [ 21,  21,  21],
        [ 14,  14,  14],
        ...,
        [228, 228, 228],
        [228, 228, 228],
        [228, 228, 228]],

       [[ 37,  37,  37],
        [ 21,  21,  21],
        [ 12,  12,  12],
        ...,
        [229, 229, 229],
        [231, 231, 231],
        [231, 231, 231]],

       [[ 28,  28,  28],
        [ 19,  19,  19],
        [ 14,  14,  14],
        ...,
        [228, 228, 228],
        [229, 229, 229],
        [230, 230, 230]],

       ...,

       [[230, 230, 230],
        [230, 230, 230],
        [230, 230, 230],
        ...,
        [226, 226, 226],
        [227, 227, 227],
        [229, 229, 229]],

       [[229, 229, 229],
        [228, 228, 228],
        [231, 231, 231],
        ...,
        [227, 227, 227],
        [227, 227, 227],
        [227, 227, 227]],

       [[231, 231, 231],
        [228, 228, 228],
        [231, 231, 231],
        ...,
        [227, 227, 227],
        [226, 226, 226],
        [226, 226, 226]

In [22]:
print(X[0].shape())

TypeError: 'tuple' object is not callable

In [3]:
import numpy as np
X_mean = np.mean(X_train)
X_std = np.std(X_train)

def standardize(x):
    return (x - X_mean) / X_std

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/_methods.py:164: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)


TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

In [18]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Lambda, MaxPooling2D, Reshape
from keras.layers import Dense, Dropout, Flatten
from keras.layers import BatchNormalization
from numpy import reshape

model = Sequential()

# model.add(Lambda(standardize, input_shape=(28, 28, 1)))

# model.add(Reshape((500, 500, 3), input_shape=(None, 500, 500, 3)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu", input_shape=(500, 500, 3)))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(filters=256, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Flatten())

model.add(Dense(512, activation="relu"))
model.add(Dense(1, activation="linear"))  # 1 output neuron for regression

model.compile(loss="mean_squared_error", optimizer="adam", metrics=["mae"])  # Using MSE loss for regression

model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_22 (Conv2D)          (None, 498, 498, 64)      1792      
                                                                 
 conv2d_23 (Conv2D)          (None, 496, 496, 64)      36928     
                                                                 
 max_pooling2d_13 (MaxPoolin  (None, 248, 248, 64)     0         
 g2D)                                                            
                                                                 
 batch_normalization_12 (Bat  (None, 248, 248, 64)     256       
 chNormalization)                                                
                                                                 
 conv2d_24 (Conv2D)          (None, 246, 246, 128)     73856     
                                                                 
 conv2d_25 (Conv2D)          (None, 244, 244, 128)    

In [20]:
import tensorflow as tf
loss_fn = tf.keras.losses.MeanSquaredError()
predictions = model(X_train[:1]).numpy()
loss = loss_fn(y_train[:1], predictions).numpy()

ValueError: Input 0 of layer "sequential_6" is incompatible with the layer: expected shape=(None, 500, 500, 3), found shape=(500, 500, 3)

In [ ]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=30)

In [ ]:
model.evaluate(X_test, y_test, verbose=2)